In [1]:
#| default_exp reading
%load_ext autoreload
%autoreload 2

import sys
from fastcore.xtras import Path
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Leitura
> Este módulo concentra funções para a leitura dos diversos arquivos que compõe a base de dados

* `STEL` - Serviços Privados de Telecomunicações
* `RADCOM` - Serviço de Radiodifusão Comunitária
* `MOSAICO` - Demais serviços de Radiodifusão, e.g. TV, RTV, FM, AM, etc.
* `AERONAUTICA` - Consolidação de diversas bases de dados públicas da aeronáutica 

In [2]:
#| export
import os
from typing import Union
from pathlib import Path

import pandas as pd
import pyodbc
from pymongo import MongoClient
from dotenv import load_dotenv

from extracao.updates import (
    update_srd,
    update_stel,
    update_radcom,
    update_base,
    update_telecom,
    update_aero,
)

from extracao.icao import get_icao
from extracao.aisgeo import get_aisg
from extracao.aisweb import get_aisw
from extracao.redemet import get_redemet
from extracao.format import _read_df

load_dotenv()

True

## MOSAICO
> O mosaico atualmente é composto por 2 bases complementares originárias de um banco `MongoDB`: 

-  `Estações`
- `Plano Básico`

In [7]:
#| export
def read_srd(
    folder: Union[str, Path],  # Pasta onde ler/salvar os dados
    conn: MongoClient = None,  # Objeto de Conexão com o banco MongoDB, atualiza os dados caso válido
) -> pd.DataFrame:  # Dataframe com os dados do mosaico
    """Lê o banco de dados salvo localmente do MOSAICO e opcionalmente o atualiza."""
    return update_srd(conn, folder) if conn else _read_df(folder, "srd")


In [4]:
folder = Path.cwd().parent / 'dados'

In [5]:
from extracao.constants import MIN_LAT, MAX_LAT, MIN_LONG, MAX_LONG
from pandas_profiling import ProfileReport

In [8]:
mosaico = read_mosaico(folder)

In [9]:
for c in ['Latitude', 'Longitude']:
    mosaico[c] = mosaico[c].astype('float')

In [10]:
mos_bad_coords = mosaico[~(mosaico.Latitude.between(MIN_LAT, MAX_LAT) | mosaico.Longitude.between(MIN_LONG, MAX_LONG))]
mos_bad_coords

,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,Num_Serviço,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade
0,207.0,REDE DE COMUNICACOES ACREANA LTDA,50442889933,<NA>,Cruzeiro do Sul,1200203,AC,NaN,NaN,A,248,<NA>,6000.0,<NA>,TV-C1,MOS,1
1,539.0,X-MEDIAGROUP S.A.,50410887137,<NA>,Mâncio Lima,1200336,AC,NaN,NaN,C,248,<NA>,6000.0,<NA>,TV-C1,MOS,1
16,593.0,FUNDACAO EDUCACIONAL E CULTURAL DAS AGUAS QUENTES,50403401330,<NA>,Caldas Novas,5204508,GO,NaN,NaN,B,248,<NA>,6000.0,<NA>,TV-C1,MOS,1
17,665.0,GUARANI RADIODIFUSAO LTDA,50410887218,<NA>,Caldas Novas,5204508,GO,NaN,NaN,C,248,<NA>,6000.0,2023-12-31,TV-C2,MOS,1
25,551.0,OCAN COMUNICACAO DIGITAL SE LTDA,50410887307,1004428283,Santa Quitéria do Maranhão,2110104,MA,NaN,NaN,C,248,<NA>,6000.0,2023-12-31,TV-C3,MOS,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30359,635.0,REDE RONDONIA DE COMUNICACAO LTDA,50444067248,<NA>,Codajás,1301308,AM,NaN,NaN,C,801,<NA>,5700.0,<NA>,TV-C1,MOS,1
30360,629.0,REDE RONDONIA DE COMUNICACAO LTDA,50444067400,<NA>,Parintins,1303403,AM,NaN,NaN,C,801,<NA>,5700.0,<NA>,TV-C1,MOS,1
30361,91.1,SISTEMA NOROESTE DE COMUNICACAO LTDA EPP,50407817506,1007790129,Piacatu,3537701,SP,NaN,NaN,C,230,<NA>,256.0,2029-01-08,FM-C4,MOS,1
30362,557.0,KAKE TV LTDA,50444198776,<NA>,Espigão D'Oeste,1100098,RO,NaN,NaN,C,801,<NA>,5700.0,2043-01-12,TV-C2,MOS,1


In [11]:
print(f"A base de radiodifusão do Mosaico possui {len(mos_bad_coords)} registros de coordenadas inválidas do total de {len(mosaico)}")

A base de radiodifusão do Mosaico possui 15206 registros de coordenadas inválidas do total de 30364


In [12]:
mosaico['Frequência'] = mosaico['Frequência'].astype('category')
mosaico_profile = ProfileReport(mosaico, config_file='report_config.yaml', title='MOSAICO')
mosaico_profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

## LICENCIAMENTO
> A Base licenciamento é composta de serviços privados de telecomunicações, tanto antigos migrados do STEL como novos para um novo banco de dados `MongoDB`

In [13]:
#| export
def read_telecom(
    folder: Union[str, Path],  # Pasta onde ler/salvar os dados
    conn: MongoClient = None,  # Objeto de Conexão com o banco MongoDB, atualiza os dados caso válido
) -> pd.DataFrame:  # Dataframe com os dados do mosaico
    """Lê o banco de dados salvo localmente do LICENCIAMENTO e opcionalmente o atualiza."""
    return update_telecom(conn, folder) if conn else _read_df(folder, "telecom")


In [16]:
#|eval: false
#|code_fold: true
telecom = read_telecom(folder)


In [18]:
#|eval: false
for c in ['Latitude', 'Longitude']:
    telecom[c] = telecom[c].astype('float')

,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,Num_Serviço,Classe_Emissão,Largura_Emissão(kHz),Validade_RF,Status,Fonte,Multiplicidade
68,368.5625,Cricom do Brasil Ltda,50402602889,1002369336,Curitiba,4106902,PR,-49.255278,-25.408333,ML,011,-1,-1,2035-11-11,L,MOS,603
69,368.6125,Cricom do Brasil Ltda,50402602889,1002369336,Curitiba,4106902,PR,-49.255278,-25.408333,ML,011,-1,-1,2035-11-11,L,MOS,603
70,368.7625,Cricom do Brasil Ltda,50402602889,1002369336,Curitiba,4106902,PR,-49.255278,-25.408333,ML,011,-1,-1,2035-11-11,L,MOS,603
71,368.7875,Cricom do Brasil Ltda,50402602889,1002369336,Curitiba,4106902,PR,-49.255278,-25.408333,ML,011,-1,-1,2035-11-11,L,MOS,603
72,368.8375,Cricom do Brasil Ltda,50402602889,1002369336,Curitiba,4106902,PR,-49.255278,-25.408333,ML,011,-1,-1,2035-11-11,L,MOS,603
73,373.1375,Cricom do Brasil Ltda,50402602889,1002370997,Curitiba,4106902,PR,-49.255278,-25.408333,ML,011,-1,-1,2035-11-11,L,MOS,602
74,373.1875,Cricom do Brasil Ltda,50402602889,1002370997,Curitiba,4106902,PR,-49.255278,-25.408333,ML,011,-1,-1,2035-11-11,L,MOS,602
75,373.2375,Cricom do Brasil Ltda,50402602889,1002370997,Curitiba,4106902,PR,-49.255278,-25.408333,ML,011,-1,-1,2035-11-11,L,MOS,602
76,373.2875,Cricom do Brasil Ltda,50402602889,1002370997,Curitiba,4106902,PR,-49.255278,-25.408333,ML,011,-1,-1,2035-11-11,L,MOS,602
77,373.3375,Cricom do Brasil Ltda,50402602889,1002370997,Curitiba,4106902,PR,-49.255278,-25.408333,ML,011,-1,-1,2035-11-11,L,MOS,602


In [ ]:

telecom[~(telecom.Latitude.between(MIN_LAT, MAX_LAT) | telecom.Longitude.between(MIN_LONG, MAX_LONG))]

In [ ]:
telecom['Frequência'] = telecom['Frequência'].astype('category')
telecom_profile = ProfileReport(telecom, config_file='report_config.yaml', title='Telecomunicações')
telecom_profile.to_notebook_iframe()

##  RADCOM

In [8]:
#| export
def read_radcom(
    folder: Union[str, Path],  # Pasta onde ler/salvar os dados
    conn: pyodbc.Connection = None,  # Objeto de conexão de banco, atualiza os dados caso válido
) -> pd.DataFrame:  # Dataframe com os dados de RADCOM
    """Lê o banco de dados salvo localmente de RADCOM. Opcionalmente o atualiza pelo Banco de Dados ANATELBDRO05 caso `update = True` ou não exista o arquivo local"""
    return update_radcom(conn, folder) if conn else _read_df(folder, "radcom")


In [9]:
#|eval: false
#|code_fold: true
radcom = read_radcom(folder)
# radcom['Frequência'] = radcom['Frequência'].astype('category')
# radcom_profile = ProfileReport(radcom, config_file='report_config.yaml', title='RADCOM')
# radcom_profile.to_notebook_iframe()

## STEL

In [11]:
#| export
def read_stel(
    folder: Union[str, Path],  # Pasta onde ler/salvar os dados
    conn: pyodbc.Connection = None,  # Objeto de conexão de banco. Atualiza os dados caso válido
) -> pd.DataFrame:  # Dataframe com os dados do STEL
    """Lê o banco de dados salvo localmente do STEL.
     Opcionalmente o atualiza pelo Banco de Dados ANATELBDRO05
    caso `update = True` ou não exista o arquivo local"""
    return update_stel(conn, folder) if conn else _read_df(folder, "stel")


In [12]:
#|eval: false
#|code_fold: true
stel = read_stel(folder)
# stel['Frequência'] = stel['Frequência'].astype('category')
# stel_profile = ProfileReport(stel, config_file='report_config.yaml', title='STEL')
# stel_profile.to_notebook_iframe()

In [30]:
def valida_coords(
    conn: pyodbc.Connection,  # Objeto de conexão de banco
    row: pd.Series,  # Linha de um DataFrame
) -> tuple:  # DataFrame com dados do município
    """Valida os dados de coordenadas e município em `row` no polígono dos municípios em banco corporativo"""
    
    sql = SQL_VALIDA_COORD.format(
        row.Longitude, row.Latitude, row.Código_Município
    )
    crsr = conn.cursor()
    crsr.execute(sql)
    result = crsr.fetchone()
    if result is None:
        return (row.Código_Município, row.Longitude, row.Latitude, -1)
    elif result.COORD_VALIDA == 1:
        return result
    else:
        return (
            result.NO_MUNICIPIO,
            result.NU_LONGITUDE,
            result.NU_LATITUDE,
            result.COORD_VALIDA,
        )

## Bases Externas da Aeronáutica

In [20]:
#| export
def read_icao(
    folder: Union[str, Path],  # Pasta onde ler/salvar os dados
    update: bool = False,  # Atualiza os dados caso `True`
) -> pd.DataFrame:  # Dataframe com os dados do ICAO
    """Lê a base de dados do Frequency Finder e Canalização VOR/ILS/DME"""
    return get_icao if update else _read_df(folder, "icao")


In [21]:
#| export
def read_aisw(
    folder: Union[str, Path],  # Pasta onde ler/salvar os dados
    update: bool = False,  # Atualiza os dados caso `True`
) -> pd.DataFrame:  # Dataframe com os dados do AISWEB
    """Fontes da informação: AISWEB, REDEMET, Ofício nº 2/SSARP/14410 e Canalização VOR/ILS/DME."""
    return get_aisw() if update else _read_df(folder, "aisw")


In [22]:
#| export
def read_aisg(
    folder: Union[str, Path],  # Pasta onde ler/salvar os dados
    update: bool = False,  # Atualiza os dados caso `True`
) -> pd.DataFrame:  # Dataframe com os dados do GEOAISWEB
    """Fontes da informação: GEOAISWEB, REDEMET, Ofício nº 2/SSARP/14410 e Canalização VOR/ILS/DME."""
    return get_aisg() if update else _read_df(folder, "aisg")


In [23]:
#| export
def read_redemet(
    folder: Union[str, Path],  # Pasta onde ler/salvar os dados
    update: bool = False,  # Atualiza os dados caso `True`
) -> pd.DataFrame:  # Dataframe com os dados do AISWEB
    """Fontes da informação: AISWEB, REDEMET, Ofício nº 2/SSARP/14410 e Canalização VOR/ILS/DME."""
    return get_redemet() if update else _read_df(folder, "redemet")


In [24]:
#| export
def read_aero(
    folder: Union[str, Path],  # Pasta onde ler/salvar os dados
    update: bool = False,  # Atualiza os dados caso `True`
) -> pd.DataFrame:  # Dataframe com os dados mesclados das 3 bases da Aeronáutica anteriores
    """Lê os arquivos de dados da aeronáutica e retorna os registros comuns e únicos"""
    return update_aero(folder) if update else _read_df(folder, "aero")


In [25]:
#| eval: false
aero = read_aero(folder)
aero['Frequency'] = aero['Frequency'].astype('category')
aero_profile = ProfileReport(aero, config_file='report_config.yaml', title='AERONAUTICA')
aero_profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

## Base Consolidada

In [26]:
#| export
def read_base(
    folder: Union[str, Path],
    conn: pyodbc.Connection = None,  # Objeto de conexão do banco SQL Server
    clientMongoDB: MongoClient = None,  # Objeto de conexão do banco MongoDB
) -> pd.DataFrame:
    """Lê a base de dados e opcionalmente a atualiza antes da leitura casos as conexões de banco sejam válidas"""
    return (
        update_base(conn, clientMongoDB, folder)
        if all([conn, clientMongoDB])
        else _read_df(folder, "base")
    )

In [27]:
#|eval: false
#|code_fold: true
base = read_base(folder)
base['Frequência'] = base['Frequência'].astype('category')
base_profile = ProfileReport(base, config_file='report_config.yaml', title='Base Consolidada')
base_profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]